# Import

In [ ]:
%pip install pandas
%pip install beautifulsoup4 python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Dados

In [ ]:
import pandas as pd

df_low_level = pd.read_csv('info_music_dataset/acousticbrainz-lowlevel.csv')
df_rythm = pd.read_csv('info_music_dataset/acousticbrainz-rythm.csv')
df_tonal = pd.read_csv('info_music_dataset/acousticbrainz-tonal.csv')

# Funções

In [2]:
import requests
import uuid
import time
import traceback
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import musicbrainzngs as mb
import re
import threading

mb.set_useragent("trilha-filmes-analisador", "1.0", "email@exemplo.com")

def gerar_id_musica():
    return str(uuid.uuid4())

def buscar_trilha_soundtracknet_bs4(filme):
    BASE_URL = "https://www.soundtrack.net"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
    try:
        # Busca pelo filme
        busca_url = f"{BASE_URL}/search/?q={filme.replace(' ', '+')}"
        response = requests.get(busca_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Busca o link da página do filme
        movie_link = None
        for card in soup.find_all(class_=['card', 'movie', 'search-result']):
            link = card.find('a', href=lambda x: x and '/movie/' in x)
            if link:
                movie_link = urljoin(BASE_URL, link['href'])
                break
        
        # Se não encontrou o filme, tenta buscar na seção de álbuns
        if not movie_link:
            for card in soup.find_all(class_=['card', 'album', 'search-result']):
                link = card.find('a', href=lambda x: x and '/album/' in x)
                if link:
                    movie_link = urljoin(BASE_URL, link['href'])
                    break
        
        if not movie_link:
            return []
        
        # Acessa a página do filme
        response = requests.get(movie_link, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        musicas = []
        
        # Método 1: Busca na seção "Song Credits"
        song_credits_section = soup.find('h2', string='Song Credits')
        if song_credits_section:
            song_list = song_credits_section.find_next('ul', class_='list-group')
            if song_list:
                for item in song_list.find_all('li', class_='list-group-item'):
                    song_title = item.find('span', class_='songtitle')
                    if song_title:
                        song_title = song_title.get_text(strip=True).strip('"')
                        creditos_completos = item.get_text().replace(f'"{song_title}"', '').strip()
                        artista_principal = extrair_artista_principal(creditos_completos)
                        musicas.append({
                            'titulo': song_title,
                            'artista_principal': artista_principal,
                            'creditos_completos': creditos_completos
                        })
        
        # Método 2: Busca na seção "Soundtrack Albums" e vai para a página do álbum
        if not musicas:
            album_links = []
            soundtrack_section = soup.find('h2', string='Soundtrack Albums')
            
            if soundtrack_section:
                album_cards = soundtrack_section.find_next('div', class_='row').find_all('div', class_='card')
                for card in album_cards:
                    link = card.find('a', href=lambda x: x and '/album/' in x)
                    if link:
                        album_links.append(urljoin(BASE_URL, link['href']))
            
            # Também verifica links diretos para álbuns na página
            outros_links = soup.find_all('a', href=lambda x: x and '/album/' in x)
            for link in outros_links:
                album_url = urljoin(BASE_URL, link['href'])
                if album_url not in album_links:
                    album_links.append(album_url)
            
            # Visita cada página de álbum
            for album_url in album_links:
                try:
                    album_response = requests.get(album_url, headers=headers, timeout=10)
                    album_response.raise_for_status()
                    album_soup = BeautifulSoup(album_response.text, 'html.parser')
                    
                    # Busca tabelas de faixas
                    track_tables = album_soup.find_all('table', class_='tracks')
                    for table in track_tables:
                        rows = table.find_all('tr')
                        for row in rows:
                            if 'table-header' in row.get('class', []):
                                continue
                            
                            # Extrai informações da faixa
                            title_cell = row.find('td', class_='title')
                            if title_cell:
                                song_title = title_cell.get_text(strip=True)
                                
                                # Procura informações de artista
                                artist_cell = row.find('td', class_='performer')
                                artist = artist_cell.get_text(strip=True) if artist_cell else ""
                                
                                # Informações adicionais
                                composer_cell = row.find('td', class_='composer')
                                composer = composer_cell.get_text(strip=True) if composer_cell else ""
                                
                                creditos_completos = f"Performer: {artist}, Composer: {composer}".strip()
                                artista_principal = artist if artist else extrair_artista_principal(composer)
                                
                                musicas.append({
                                    'titulo': song_title,
                                    'artista_principal': artista_principal,
                                    'creditos_completos': creditos_completos
                                })
                except Exception as e:
                    print(f"Erro ao processar álbum {album_url}: {e}")
                    continue
        
        # Método 3: Busca diretamente nas tabelas de faixas na página do filme
        if not musicas:
            track_tables = soup.find_all('table', class_='tracks')
            for table in track_tables:
                rows = table.find_all('tr')
                for row in rows:
                    if 'table-header' in row.get('class', []):
                        continue
                    
                    title_cell = row.find('td', class_='title')
                    if title_cell:
                        song_title = title_cell.get_text(strip=True)
                        
                        artist_cell = row.find('td', class_='performer')
                        artist = artist_cell.get_text(strip=True) if artist_cell else ""
                        
                        composer_cell = row.find('td', class_='composer')
                        composer = composer_cell.get_text(strip=True) if composer_cell else ""
                        
                        creditos_completos = f"Performer: {artist}, Composer: {composer}".strip()
                        artista_principal = artist if artist else extrair_artista_principal(composer)
                        
                        musicas.append({
                            'titulo': song_title,
                            'artista_principal': artista_principal,
                            'creditos_completos': creditos_completos
                        })
        
        # Verifica se há links para outras páginas com informações adicionais sobre o soundtrack
        soundtrack_links = soup.find_all('a', href=lambda x: x and '/movie/' in x and 'soundtrack' in x)
        for link in soundtrack_links:
            try:
                soundtrack_url = urljoin(BASE_URL, link['href'])
                if soundtrack_url != movie_link:  # Evita loops
                    soundtrack_response = requests.get(soundtrack_url, headers=headers, timeout=10)
                    soundtrack_response.raise_for_status()
                    soundtrack_soup = BeautifulSoup(soundtrack_response.text, 'html.parser')
                    
                    # Busca em listas de músicas
                    song_lists = soundtrack_soup.find_all('ul', class_='list-group')
                    for song_list in song_lists:
                        for item in song_list.find_all('li', class_='list-group-item'):
                            song_title = item.find('span', class_='songtitle')
                            if song_title:
                                song_title = song_title.get_text(strip=True).strip('"')
                                creditos_completos = item.get_text().replace(f'"{song_title}"', '').strip()
                                artista_principal = extrair_artista_principal(creditos_completos)
                                
                                # Evita duplicatas
                                if not any(m['titulo'] == song_title for m in musicas):
                                    musicas.append({
                                        'titulo': song_title,
                                        'artista_principal': artista_principal,
                                        'creditos_completos': creditos_completos
                                    })
            except Exception as e:
                print(f"Erro ao processar página adicional {soundtrack_url}: {e}")
                continue
        
        # Remove duplicatas baseadas no título da música
        musicas_unicas = []
        titulos_vistos = set()
        for musica in musicas:
            if musica['titulo'] not in titulos_vistos:
                titulos_vistos.add(musica['titulo'])
                musicas_unicas.append(musica)
        
        return musicas_unicas
        
    except Exception as e:
        print(f"Erro ao buscar trilha: {e}")
        traceback.print_exc()  # Adiciona stack trace para depuração
        return []

# Função auxiliar para extrair o artista principal
def extrair_artista_principal(creditos):
    # Padrões comuns para encontrar o artista
    padroes = [
        r"[Pp]erformed by\s+([^,\.]+)",
        r"[Pp]erformer:\s+([^,\.]+)",
        r"[Ww]ritten by\s+([^,\.]+)",
        r"[Cc]omposed by\s+([^,\.]+)",
        r"[Mm]usic by\s+([^,\.]+)",
        r"[Ss]ung by\s+([^,\.]+)",
        r"[Bb]y\s+([^,\.]+)"
    ]
    
    for padrao in padroes:
        match = re.search(padrao, creditos)
        if match:
            return match.group(1).strip()
            
    # Se não encontrar um padrão específico, tenta extrair o primeiro nome após dois pontos
    match = re.search(r":\s+([^,\.]+)", creditos)
    if match:
        return match.group(1).strip()
        
    return ""

def flatten_dict(d, parent_key='', sep='_'):
    """
    Função auxiliar que planifica um dicionário aninhado (usada para os dados do AcousticBrainz)
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def buscar_info_acousticbrainz(titulo, artista):
    try:
        resultados = mb.search_recordings(recording=titulo, artist=artista, limit=1)
        if not resultados['recording-list']:
            print(f"❌ MBID não encontrado para: {titulo} - {artista}")
            return None

        recording = resultados['recording-list'][0]
        mbid = recording.get('id')
        if not mbid:
            print(f"❌ MBID ausente para: {titulo} - {artista}")
            return None

        features = {}

        # ---- LOW-LEVEL ----
        dados_low = df_low_level[df_low_level['mbid'] == mbid]
        if not dados_low.empty:
            dados_low = dados_low.iloc[0]
            features.update({
                'submission_offset_low_level': dados_low.get('submission_offset'),
                'average_loudness': dados_low.get('average_loudness'),
                'dynamic_complexity': dados_low.get('dynamic_complexity'),
                'mfcc_zero_mean': dados_low.get('mfcc_zero_mean')
            })
        # else:
        #    print(f"⚠ MBID {mbid} não encontrado no df_low_level")

        # ---- RYTHM ----
        dados_rythm = df_rythm[df_rythm['mbid'] == mbid]
        if not dados_rythm.empty:
            dados_rythm = dados_rythm.iloc[0]
            features.update({
                'submission_offset_rythm': dados_rythm.get('submission_offset'),
                'bpm': dados_rythm.get('bpm'),
                'bpm_first_peak_mean': dados_rythm.get('bpm_histogram_first_peak_bpm.mean'),
                'bpm_first_peak_median': dados_rythm.get('bpm_histogram_first_peak_bpm.median'),
                'bpm_second_peak_mean': dados_rythm.get('bpm_histogram_second_peak_bpm.mean'),
                'bpm_second_peak_median': dados_rythm.get('bpm_histogram_second_peak_bpm.median'),
                'danceability': dados_rythm.get('danceability'),
                'onset_rate': dados_rythm.get('onset_rate')
            })
        # else:
        #    print(f"⚠ MBID {mbid} não encontrado no df_rythm")

        # ---- TONAL ----
        dados_tonal = df_tonal[df_tonal['mbid'] == mbid]
        if not dados_tonal.empty:
            dados_tonal = dados_tonal.iloc[0]
            features.update({
                'submission_offset_tonal': dados_tonal.get('submission_offset'),
                'key': dados_tonal.get('key_key'),
                'scale': dados_tonal.get('key_scale'),
                'tuning_frequency': dados_tonal.get('tuning_frequency'),
                'tuning_equal_tempered_deviation': dados_tonal.get('tuning_equal_tempered_deviation')
            })
        # else:
        #    print(f"⚠ MBID {mbid} não encontrado no df_tonal")

        return features if features else None

    except Exception as e:
        print(f"❌ Erro geral em buscar_info_acousticbrainz para {titulo}: {e}")
        return None

def analisar_varias_trilhas(lista_filmes, nomearquivo, salvar_csv=True):

    df_geral = []

    for filme in lista_filmes:
        print(f"\n🎥 Analisando trilha sonora de: {filme}")
        trilha = buscar_trilha_soundtracknet_bs4(filme)
        print(f"🎬 {filme}: {len(trilha)} músicas encontradas.")
        
        if not trilha:
            print("⚠ Nenhuma música encontrada para este filme.")
            continue
        
        for musica in trilha:
            musica_id = gerar_id_musica()
            ab_info = buscar_info_acousticbrainz(musica['titulo'], musica['artista_principal'])

            if not ab_info:
                print(f"⚠ Não encontrado no AcousticBrainz: {musica['titulo']}")
                ab_info = {}

            linha = {
                'id_musica': musica_id,
                'filme': filme,
                'titulo_musica': musica['titulo'],
                'artista_principal': musica['artista_principal'],
                'creditos_completos': musica['creditos_completos'],
                **ab_info
            }
            df_geral.append(linha)
            time.sleep(1)  # respeitar a API
    
    df_final = pd.DataFrame(df_geral)

    if salvar_csv and not df_final.empty:
        df_final.to_csv(nomearquivo, index=False, encoding='utf-8-sig')
        print(f"\n✅ CSV final salvo como {nomearquivo}")
    elif df_final.empty:
        print("⚠ Nenhuma música foi encontrada em nenhum dos filmes.")
    
    return df_final

def dividir_em_partes(lista, x):
    tamanho = len(lista)
    return [lista[i * tamanho // x : (i + 1) * tamanho // x] for i in range(x)]

def analisar_trilhas_thread(lista_filmes, nomearquivo, num_threads=10, salvar_csv=True):
    """
    Função principal para analisar a trilha sonora de um filme.
    """
    listas = dividir_em_partes(lista_filmes, num_threads)

    threads = []
    for i in range (num_threads):
        print(f"\n🔄 Analisando parte {i + 1} de {num_threads}...")
        t = threading.Thread(target=analisar_varias_trilhas, args=(listas[i], f"{nomearquivo}_{i + 1}.csv"))
        threads.append(t)
        t.start()
    
    for i in range (num_threads):
        threads[i].join()
        print(f"✅ Parte {i + 1} concluída.")
    print("\n🔄 Todas as partes concluídas.")


In [3]:
def coletar_filmes_wikipedia_tabela(url):
    resposta = requests.get(url)
    soup = BeautifulSoup(resposta.text, 'html.parser')

    filmes = []

    # Procura por todas as tabelas com a classe wikitable
    tabelas = soup.find_all('table', class_='wikitable')

    for tabela in tabelas:
        linhas = tabela.find_all('tr')[1:]  # Pula o cabeçalho
        for linha in linhas:
            colunas = linha.find_all('td')
            if colunas:
                titulo = colunas[0].get_text(strip=True)
                filmes.append(titulo)

    return filmes

def coletar_filmes_wikipedia_lista(url):
    resposta = requests.get(url)
    soup = BeautifulSoup(resposta.text, 'html.parser')

    filmes = []

    # A estrutura da página tem listas <ul> com itens <li> onde estão os títulos dos filmes
    listas = soup.find_all('ul')

    for ul in listas:
        for li in ul.find_all('li', recursive=False):
            # Ignora entradas que contenham links para listas ou artigos grandes
            if li.find('i'):
                titulo = li.find('i').get_text(strip=True)
                filmes.append(titulo)

    return filmes

In [4]:
def atualiza_cache(filmes_guarda):
    # Carrega o cache existente
    cached_movies = load_cache()
    
    if cached_movies:
        new_movies = filtrar_filmes(filmes_guarda, cached_movies)

    # Adiciona novos filmes ao cache (evitando duplicatas)
        updated_cache = list(set(cached_movies + new_movies))
    else:
        updated_cache = filmes_guarda
        new_movies = filmes_guarda
    
    # Retorna a amostra solicitada
    return new_movies, updated_cache

def filtrar_filmes(lista_filmes_base, lista_filmes_remover):
    # Garante que as comparações sejam feitas com padronização de maiúsculas/minúsculas
    if (not lista_filmes_base or not lista_filmes_remover):
        return lista_filmes_base
    remover_set = set(f.strip().lower() for f in lista_filmes_remover)
    return [f for f in lista_filmes_base if f.strip().lower() not in remover_set]

def load_cache(caminho_arquivo='filmesJaUsados.txt'):
    filmes = []
    with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
        conteudo = arquivo.read()

        # Remove quebras de linha e espaços desnecessários
        conteudo = conteudo.replace('\n', ' ').replace('\r', ' ').strip()

        # Divide por vírgula
        titulos = conteudo.split(',')

        for titulo in titulos:
            # Remove aspas e espaços extras
            titulo_limpo = titulo.strip().strip('"').strip()
            if titulo_limpo:  # Evita strings vazias
                filmes.append(titulo_limpo)
    
    return filmes

def save_cache(movie_list, cache_file='filmesJaUsados.txt'):
    """Salva a lista de filmes no cache no formato especificado"""
    with open(cache_file, 'w', encoding='utf-8') as f:
        # Formata como "filme1","filme2",...
        formatted_movies = ','.join(f'"{movie}"' for movie in movie_list)
        f.write(formatted_movies)

# Acao

In [ ]:
for ano in [1970, 1980, 1990, 2000, 2010, 2020]:
    url = f"https://en.wikipedia.org/wiki/List_of_action_films_of_the_{ano}s"
    filmesacao = coletar_filmes_wikipedia_tabela(url)
    print(len(filmesacao), "filmes encontrados na Wikipedia.")
    filmesacao, cache = atualiza_cache(filmesacao)
    print(len(filmesacao))
    df_trilhas = analisar_varias_trilhas(filmesacao, f'acao{ano}s_pt1.csv')
    save_cache(cache)

253 filmes encontrados na Wikipedia.


# Drama

In [ ]:
for ano in [1970, 1980, 1990, 2000, 2010, 2020]:
    url = f'https://en.wikipedia.org/wiki/List_of_drama_films_of_the_{ano}s'
    filmesdrama = coletar_filmes_wikipedia_lista(url)
    print(len(filmesdrama), "filmes encontrados na Wikipedia.")
    filmesdrama1, cache = atualiza_cache(filmesdrama)
    print(len(filmesdrama1))
    df_trilhas = analisar_varias_trilhas(filmesdrama1, f'dramas{ano}s.csv')
    save_cache(cache)

77 filmes encontrados na Wikipedia.


# Ficção Científica

In [ ]:
for ano in [1970, 1980, 1990, 2000, 2010, 2020]:
    url = f'https://en.wikipedia.org/wiki/List_of_science_fiction_films_of_the_{ano}s'
    filmesficcao = coletar_filmes_wikipedia_tabela(url) 
    print(len(filmesficcao), "filmes encontrados na Wikipedia.")
    filmesficcao, cache = atualiza_cache(filmesficcao)
    print(len(filmesficcao))
    df_trilhas = analisar_varias_trilhas(filmesficcao, 'ficcao1970s_pt1.csv')
    save_cache(cache)

258 filmes encontrados na Wikipedia.


# Animação

In [ ]:
for ano in range(1980, 2023):
    url = f'https://en.wikipedia.org/wiki/List_of_animated_feature_films_of_{ano}'
    filmesanimacao = coletar_filmes_wikipedia_tabela(url)
    print(len(filmesanimacao), "filmes encontrados na Wikipedia.")

    filmesanimacao, cache = atualiza_cache(filmesanimacao)
    print(len(filmesanimacao))
    df_trilhas = analisar_varias_trilhas(filmesanimacao, f'animacao{ano}.csv')
    save_cache(cache)

53 filmes encontrados na Wikipedia.
53

🎥 Analisando trilha sonora de: 15 Sonyeon Uju Pyoryugi15소년 우주 표류기(15 Boy Space Drifter)
🎬 15 Sonyeon Uju Pyoryugi15소년 우주 표류기(15 Boy Space Drifter): 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: 3000 Leagues in Search of Mother母をたずねて三千里(Haha o Tazunete Sanzenri)
🎬 3000 Leagues in Search of Mother母をたずねて三千里(Haha o Tazunete Sanzenri): 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Animalympics
🎬 Animalympics: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Back to the Forestのどか森の動物大作戦(Nodoka Mori no Doubutsu Daisakusen)
🎬 Back to the Forestのどか森の動物大作戦(Nodoka Mori no Doubutsu Daisakusen): 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Be Forever Yamatoヤマトよ永遠に(Yamato yo Towa ni)
🎬 Be Forever Yamatoヤマトよ永遠に(Yamato yo Towa ni): 0 músicas encontrada

# Crime

In [ ]:
for ano in range(1980,2000) ++ [2000, 2010, 2020]:
    url = f'https://en.wikipedia.org/wiki/List_of_crime_films_of_{ano}'
    filmescrime = coletar_filmes_wikipedia_tabela(url)
    print(len(filmescrime), "filmes encontrados na Wikipedia.")

    filmescrime, cache = atualiza_cache(filmescrime)
    print(len(filmescrime))
    df_trilhas = analisar_varias_trilhas(filmescrime, f'crime{ano}.csv')
    save_cache(cache)

0 filmes encontrados na Wikipedia.
0


NameError: name 'pd' is not defined

# Suspense

In [ ]:
for ano in [1970, 1980, 1990, 2000, 2010, 2020]:
    url = f'https://en.wikipedia.org/wiki/List_of_thriller_films_of_the_{ano}s'
    filmesthriller = coletar_filmes_wikipedia_tabela(url)
    print(len(filmesthriller), "filmes encontrados na Wikipedia.")

    filmesthriller, cache = atualiza_cache(filmesthriller)
    print(len(filmesthriller))
    df_trilhas = analisar_varias_trilhas(filmesthriller, f'thriller{ano}s_pt1.csv')
    save_cache(cache)

341 filmes encontrados na Wikipedia.
171

🎥 Analisando trilha sonora de: After Dark, My Sweet
🎬 After Dark, My Sweet: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: The Ambulance
🎬 The Ambulance: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Bad Influence
🎬 Bad Influence: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Biktima
🎬 Biktima: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Dancing Machine
🎬 Dancing Machine: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Desperate Hours
🎬 Desperate Hours: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Hidden Agenda
🎬 Hidden Agenda: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: 

# Mais filmes

In [6]:
filmes = [
    "Pulp Fiction", "The Lion King", "Forrest Gump", "Titanic", "Gladiator", "Star Wars: A New Hope",
    "The Lord of the Rings: The Fellowship of the Ring", "Amélie", "Black Swan", "Trainspotting",
    "2001: A Space Odyssey", "Interstellar", "Fight Club", "Rocky", "Dirty Dancing",
    "Frozen", "La La Land", "Whiplash", "The Phantom of the Opera", "The Godfather",
    "Blade Runner", "Tron: Legacy", "Requiem for a Dream", "Drive", "Pan’s Labyrinth",
    "Dune", "Inception", "Joker", "The Dark Knight", "The Social Network",
    "The Great Gatsby", "Saving Private Ryan", "Léon: The Professional", "Donnie Darko",
    "The Nightmare Before Christmas", "Harry Potter and the Sorcerer’s Stone", "The Avengers",
    "Batman", "Birdman", "Moonlight", "The Artist", "Her", "The Shape of Water", "The Pianist",
    "The Da Vinci Code", "The Hateful Eight", "Kill Bill: Vol. 1", "Brokeback Mountain",
    "The Last Temptation of Christ", "RocknRolla", "The Grand Budapest Hotel", "Juno",
    "The Good, the Bad and the Ugly", "The Exorcist", "Psycho", "Goodfellas",
    "Once Upon a Time in Hollywood", "The Wolf of Wall Street", "Tenet", "The Departed",
    "The Curious Case of Benjamin Button", "Life of Pi", "The Greatest Showman",
    "Bohemian Rhapsody", "Rocketman", "Soul", "Encanto", "Moana", "Toy Story", "WALL·E",
    "Ratatouille", "The Incredibles", "Up", "Fantastic Mr. Fox", "Shutter Island",
    "The Matrix", "The Big Lebowski", "American Beauty", "Into the Wild", "Selma",
    "12 Years a Slave", "The King's Speech", "The Imitation Game", "The Theory of Everything",
    "Arrival", "Dunkirk", "The Revenant", "Noah", "The Lighthouse", "Ad Astra", "Roma",
    "Black Panther", "V for Vendetta", "The Untouchables", "Moulin Rouge!", "Chicago",
    "Cats", "Les Misérables", "Across the Universe"
]

print(len(filmes))
analisar_trilhas_thread(filmes, 'trilhas_sonoras_gpt2.csv')

99

🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: Pulp Fiction

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Trainspotting

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: The Godfather

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: The Social Network

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Moonlight

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: RocknRolla

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: The Departed

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: WALL·E

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Selma

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Ad Astra
🎬 WALL·E: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Ratatouille
🎬 RocknRolla: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: The Grand Buda

In [7]:
more_movies_with_great_soundtracks = [
    "Amadeus", "The Mission", "Braveheart", "The Last of the Mohicans", "Pirates of the Caribbean: The Curse of the Black Pearl",
    "Schindler's List", "The Piano", "Magnolia", "Atonement", "Edward Scissorhands",
    "The Hours", "The Thin Red Line", "Crouching Tiger, Hidden Dragon", "Slumdog Millionaire", "The English Patient",
    "The Fountain", "The Rock", "Cast Away", "Pearl Harbor", "Memoirs of a Geisha",
    "The Village", "Signs", "Unbreakable", "The Sixth Sense", "Snowpiercer",
    "The Assassination of Jesse James by the Coward Robert Ford", "The New World", "The Tree of Life", "True Romance", "Heat",
    "Man of Steel", "The Batman (2022)", "Oblivion", "The Book of Eli", "The Hunger Games",
    "Cinderella (2015)", "Maleficent", "Beauty and the Beast (2017)", "Aladdin (2019)", "The Little Mermaid (1989)",
    "Finding Nemo", "Coco", "Brave", "Inside Out", "Luca",
    "The Secret Life of Walter Mitty", "The Perks of Being a Wallflower", "500 Days of Summer", "Garden State", "Begin Again",
    "August Rush", "Once", "Yesterday", "Sing Street", "Music and Lyrics",
    "A Star is Born (2018)", "School of Rock", "That Thing You Do!", "Almost Famous", "This Is Spinal Tap",
    "Walk the Line", "Ray", "The Blues Brothers", "Inside Llewyn Davis", "O Brother, Where Art Thou?",
    "Baby Driver", "Guardians of the Galaxy", "Guardians of the Galaxy Vol. 2", "Scott Pilgrim vs. the World", "Zombieland",
    "The Lego Movie", "Trolls", "Sing", "Pitch Perfect", "The Muppet Movie",
    "The Sound of Music", "West Side Story (1961)", "West Side Story (2021)", "My Fair Lady", "Fiddler on the Roof",
    "Grease", "Saturday Night Fever", "Flashdance", "Footloose", "Fame",
    "Purple Rain", "8 Mile", "Notting Hill", "Love Actually", "The Bodyguard",
    "Twilight", "Twilight: New Moon", "The Fault in Our Stars", "The Great Debaters", "Hidden Figures",
    "The Help", "Remember the Titans", "Coach Carter", "Moneyball", "Rudy",
    "A Beautiful Mind", "The Truman Show", "Patch Adams", "Dead Poets Society", "Chariots of Fire"
]

analisar_trilhas_thread(more_movies_with_great_soundtracks, 'trilhas_sonoras_gpt3.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: Amadeus

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: The Hours

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Signs

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: The Batman (2022)

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Brave

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Yesterday

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Inside Llewyn Davis

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Pitch Perfect

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Fame

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Hidden Figures
🎬 The Batman (2022): 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Oblivion
🎬 Inside Llewyn Davis: 19 músicas encontradas.
🎬 Hidden Figures: 19 músicas encontradas.
🎬 Pitch Perfect: 51 músicas encontrada

In [9]:
movies_with_great_soundtracks = [
    "Seven Samurai", "Rashomon", "Tokyo Story", "The Good, the Bad, the Weird", "Oldboy",
    "Cinema Paradiso", "Spirited Away", "Princess Mononoke", "My Neighbor Totoro", "Akira",
    "Stalker", "Solaris", "Come and See", "Metropolis", "The Cabinet of Dr. Caligari",
    "Nosferatu", "Lawrence of Arabia", "Casablanca", "Gone with the Wind", "Citizen Kane",
    "There Will Be Blood", "No Country for Old Men", "Blood Simple", "Barton Fink", "Miller's Crossing",
    "Yojimbo", "Ran", "Throne of Blood", "Harakiri", "In the Mood for Love",
    "Parasite", "Memories of Murder", "Shoplifters", "Nobody Knows", "Persona", 
    "The Seventh Seal", "Wild Strawberries", "Fanny and Alexander", "Andrei Rublev", "Ivan's Childhood",
    "Mirror", "Apocalypse Now", "Full Metal Jacket", "Barry Lyndon", "Eyes Wide Shut",
    "M", "Paths of Glory", "Dr. Strangelove", "A Clockwork Orange", "The Shining",
    "Alien", "Chinatown", "Taxi Driver", "Raging Bull", "Mulholland Drive",
    "Blue Velvet", "Lost Highway", "Eraserhead", "Vertigo", "North by Northwest",
    "Rear Window", "Breathless", "The 400 Blows", "Jules and Jim", "La Jetée",
    "The Red Shoes", "Brief Encounter", "Great Expectations", "Oliver Twist", "À bout de souffle",
    "8½", "La Dolce Vita", "Amarcord", "The Leopard", "The Conformist",
    "The Battle of Algiers", "Bicycle Thieves", "Open City", "L'Avventura", "L'Eclisse",
    "La Notte", "The Umbrellas of Cherbourg", "The Young Girls of Rochefort", "Cléo from 5 to 7", "Vivre Sa Vie",
    "The Enigma of Kaspar Hauser", "Stroszek", "Aguirre, the Wrath of God", "Fitzcarraldo", "City of God",
    "Y Tu Mamá También", "The Devil's Backbone", "The Orphanage", "Volver", "Talk to Her",
    "The Spirit of the Beehive", "Raise the Red Lantern", "Farewell My Concubine", "Chungking Express", "Happy Together"
]

analisar_trilhas_thread(movies_with_great_soundtracks, 'trilhas_sonoras_gpt4.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: Seven Samurai

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Stalker

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: There Will Be Blood

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Parasite

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Mirror

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Alien

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Rear Window

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: 8½

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: La Notte

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Y Tu Mamá También
🎬 Y Tu Mamá También: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: The Devil's Backbone
🎬 8½: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: La Dolce Vita
🎬

In [10]:
even_more_movies_with_great_soundtracks = [
    "E.T. the Extra-Terrestrial", "Back to the Future", "Beetlejuice", "Big Fish", "Catch Me If You Can",
    "Hook", "War Horse", "The Polar Express", "Home Alone", "The Goonies",
    "The NeverEnding Story", "Labyrinth", "Willow", "Coraline", "The Secret Garden",
    "Bridge to Terabithia", "Stardust", "The Golden Compass", "The Chronicles of Narnia: The Lion, the Witch and the Wardrobe", "Eragon",
    "Atlantis: The Lost Empire", "Treasure Planet", "Spirit: Stallion of the Cimarron", "The Prince of Egypt", "Anastasia",
    "The Road to El Dorado", "Sinbad: Legend of the Seven Seas", "Shrek", "Shrek 2", "Happy Feet",
    "Ice Age", "Kung Fu Panda", "Megamind", "Monsters vs. Aliens", "Madagascar",
    "How to Train Your Dragon", "How to Train Your Dragon 2", "The Croods", "Rio", "Despicable Me",
    "Minions", "Hotel Transylvania", "Cloudy with a Chance of Meatballs", "The Angry Birds Movie", "The Peanuts Movie",
    "Enchanted", "Tangled", "The Princess and the Frog", "Frozen II", "Raya and the Last Dragon",
    "Lightyear", "Turning Red", "The Mitchells vs. the Machines", "Klaus", "The Willoughbys",
    "Over the Moon", "I Lost My Body", "Wolfwalkers", "Song of the Sea", "The Breadwinner",
    "The Red Turtle", "Persepolis", "Waltz with Bashir", "Fantastic Planet", "Ghost in the Shell",
    "Akira", "Paprika", "Perfect Blue", "Your Name", "Weathering With You",
    "Spirited Away", "Princess Mononoke", "Howl's Moving Castle", "Nausicaä of the Valley of the Wind", "My Neighbor Totoro",
    "Grave of the Fireflies", "The Wind Rises", "The Tale of the Princess Kaguya", "Ponyo", "Arrietty",
    "A Silent Voice", "Children Who Chase Lost Voices", "The Garden of Words", "5 Centimeters per Second", "The Girl Who Leapt Through Time",
    "Summer Wars", "Wolf Children", "The Boy and the Beast", "Belle", "Mirai",
    "Metropolis (2001)", "Millennium Actress", "The Cat Returns", "Whisper of the Heart", "From Up on Poppy Hill",
    "When Marnie Was There", "Ocean Waves", "Only Yesterday", "The Castle of Cagliostro", "The Secret World of Arrietty"
]

analisar_trilhas_thread(even_more_movies_with_great_soundtracks, 'trilhas_sonoras_gpt5.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: E.T. the Extra-Terrestrial

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: The NeverEnding Story

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Atlantis: The Lost Empire

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Ice Age

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Minions

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Lightyear

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: The Red Turtle

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Spirited Away

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: A Silent Voice

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Metropolis (2001)
🎬 Metropolis (2001): 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Millennium Actress
🎬 Minions: 28 músicas encontradas.
🎬 E.T. the Extra-Terrestrial: 4

In [11]:
more_soundtrack_favorites = [
    "Requiem for a Dream", "Pi", "Mother!", "The Wrestler", "The Fountain", "Moon", "Ex Machina", "Annihilation", "Arrival", "Sicario",
    "Prisoners", "Blade Runner 2049", "Enemy", "Nightcrawler", "The Place Beyond the Pines", "Blue Valentine", "Drive My Car", "The Handmaiden", "Oldboy", "Memories of Murder",
    "The Wailing", "I Saw the Devil", "Burning", "Shoplifters", "Babel", "21 Grams", "Amores Perros", "Biutiful", "Y Tu Mamá También", "Roma",
    "The Motorcycle Diaries", "City of God", "Elite Squad", "Central Station", "The Sea Inside", "Talk to Her", "Volver", "The Orphanage", "Pan’s Labyrinth", "The Skin I Live In",
    "Incendies", "Polytechnique", "Enemy", "Brooklyn", "Carol", "Phantom Thread", "The Master", "There Will Be Blood", "Boogie Nights", "Magnolia",
    "Punch-Drunk Love", "Licorice Pizza", "Eyes Wide Shut", "Full Metal Jacket", "Barry Lyndon", "A Clockwork Orange", "The Shining", "Paths of Glory", "Dr. Strangelove", "Lolita",
    "The Elephant Man", "Blue Velvet", "Mulholland Drive", "Lost Highway", "Wild at Heart", "Twin Peaks: Fire Walk with Me", "The Straight Story", "Eraserhead", "Mishima: A Life in Four Chapters", "Koyaanisqatsi",
    "Baraka", "Samsara", "The Fall", "The Cell", "What Dreams May Come", "The Lovely Bones", "Coraline", "Monster House", "Frankenweenie", "ParaNorman",
    "9", "The Book of Life", "The Nightmare Before Christmas", "Corpse Bride", "The Iron Giant", "Batman: Mask of the Phantasm", "Justice League: The Flashpoint Paradox", "Spider-Man: Into the Spider-Verse", "The Lego Batman Movie", "Big Hero 6",
    "Wreck-It Ralph", "Ralph Breaks the Internet", "Bolt", "Meet the Robinsons", "The Emperor's New Groove", "Treasure Planet", "Brother Bear", "Chicken Little", "Home on the Range", "Atlantis: Milo's Return"
]

analisar_trilhas_thread(more_soundtrack_favorites, 'trilhas_sonoras_gpt6.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: Requiem for a Dream

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Prisoners

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: The Wailing

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: The Motorcycle Diaries

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Incendies

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Punch-Drunk Love

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: The Elephant Man

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Baraka

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: 9

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Wreck-It Ralph
🎬 Prisoners: 3 músicas encontradas.
🎬 Incendies: 3 músicas encontradas.
🎬 Wreck-It Ralph: 7 músicas encontradas.
🎬 Punch-Drunk Love: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Lic

In [12]:
other_movies_with_soundtracks = [
    "The Pursuit of Happyness", "Cast Away", "The Terminal", "Catch Me If You Can", "Bridge of Spies",
    "War of the Worlds", "Minority Report", "Lincoln", "The Post", "Munich",
    "Argo", "The Town", "Gone Girl", "The Girl with the Dragon Tattoo", "Zodiac",
    "Seven", "The Game", "The Curious Case of Benjamin Button", "Moneyball", "Capote",
    "Spotlight", "Green Book", "The Blind Side", "The Help", "Crash",
    "The Big Short", "Vice", "American Hustle", "Silver Linings Playbook", "Joy",
    "The Fighter", "3:10 to Yuma", "Logan", "Ford v Ferrari", "The Prestige",
    "Insomnia", "Memento", "Following", "Tenet", "Batman Begins",
    "The Dark Knight Rises", "Man of Steel", "Batman v Superman: Dawn of Justice", "Justice League", "Watchmen",
    "V for Vendetta", "Constantine", "The Matrix Reloaded", "The Matrix Revolutions", "Cloud Atlas",
    "Jupiter Ascending", "Speed Racer", "The Lovely Bones", "King Kong (2005)", "The Hobbit: An Unexpected Journey",
    "The Hobbit: The Desolation of Smaug", "The Hobbit: The Battle of the Five Armies", "Fantastic Beasts and Where to Find Them", "Fantastic Beasts: The Crimes of Grindelwald", "Fantastic Beasts: The Secrets of Dumbledore",
    "Percy Jackson & the Olympians: The Lightning Thief", "Eragon", "Inkheart", "The Spiderwick Chronicles", "City of Ember",
    "The Sorcerer’s Apprentice", "The Chronicles of Riddick", "I Am Legend", "I, Robot", "Oblivion",
    "Edge of Tomorrow", "Battle Los Angeles", "Pacific Rim", "Prometheus", "Alien: Covenant",
    "Elysium", "District 9", "Chappie", "Looper", "Source Code",
    "The Adjustment Bureau", "The Giver", "Ender’s Game", "Divergent", "Insurgent",
    "Allegiant", "The Maze Runner", "The Scorch Trials", "The Death Cure", "The Host",
    "Snow White and the Huntsman", "The Huntsman: Winter’s War", "Mirror Mirror", "Ella Enchanted", "Beastly",
    "Ink", "The Fall", "The Secret Life of Bees", "Now Is Good", "Me and Earl and the Dying Girl",
    "A Monster Calls", "The Book Thief", "Extremely Loud & Incredibly Close", "The Lovely Bones", "Remember Me"
]

analisar_trilhas_thread(other_movies_with_soundtracks, 'trilhas_sonoras_gpt7.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: The Pursuit of Happyness

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Argo

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Green Book

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: 3:10 to Yuma

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Batman v Superman: Dawn of Justice

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: The Lovely Bones

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: The Spiderwick Chronicles

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Prometheus

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Insurgent

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Beastly
🎬 3:10 to Yuma: 1 músicas encontradas.
🎬 Batman v Superman: Dawn of Justice: 4 músicas encontradas.
🎬 Argo: 15 músicas encontradas.
🎬 Insurgent: 6 músicas encontradas.
🎬 Green Book: 48 músicas encontradas.
🎬 

In [7]:
another_batch_of_movies = [
    "127 Hours", "21", "50/50", "About Time", "Adventureland", "Atonement", "Away We Go", "Begin Again", "Before Sunrise", "Before Sunset",
    "Before Midnight", "Brooklyn", "Cake", "Charlie Bartlett", "Chef", "Columbus", "Copenhagen", "Coyote Ugly", "Dan in Real Life", "Dear John",
    "Definitely, Maybe", "Don Jon", "Elizabethtown", "Everything Everywhere All at Once", "First Man", "Flipped", "Friends with Benefits", "Gifted", "Good Time", "Her",
    "Hugo", "I Love You, Man", "In the Heights", "Inside Llewyn Davis", "Juno", "La La Land", "Lady Bird", "Like Crazy", "Little Miss Sunshine", "Lords of Dogtown",
    "Love, Rosie", "Me Before You", "Midnight in Paris", "Miss Stevens", "Moonrise Kingdom", "Nick and Norah’s Infinite Playlist", "Nocturnal Animals", "Once", "One Day", "Only Lovers Left Alive",
    "Palm Springs", "Paper Towns", "Perks of Being a Wallflower", "Rocketman", "Ruby Sparks", "Run Lola Run", "Safety Not Guaranteed", "Sing Street", "Submarine", "Sunshine Cleaning",
    "Synecdoche, New York", "Take Shelter", "The Art of Getting By", "The Big Sick", "The Edge of Seventeen", "The Florida Project", "The Fundamentals of Caring", "The Half of It", "The Map of Tiny Perfect Things", "The Peanut Butter Falcon",
    "The Secret Life of Walter Mitty", "The Spectacular Now", "The Station Agent", "The Way Way Back", "The Words", "Then Came You", "To the Bone", "Under the Silver Lake", "Up in the Air", "Waking Life",
    "Warm Bodies", "We Bought a Zoo", "Whiplash", "White Bird in a Blizzard", "Wild", "Wish I Was Here", "Yes Man", "Youth", "Zathura", "Zombieland",
    "Beautiful Boy", "Boyhood", "Call Me by Your Name", "Drive", "The Bling Ring", "The Tree of Life", "Sound of Metal", "Tangerine", "Paterson", "Francis Ha"
]

analisar_trilhas_thread(another_batch_of_movies, 'trilhas_sonoras_gpt8.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: 127 Hours

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Before Midnight

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Definitely, Maybe

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Hugo

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Love, Rosie

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Palm Springs

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Synecdoche, New York

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: The Secret Life of Walter Mitty

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Warm Bodies

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Beautiful Boy
🎬 Definitely, Maybe: 25 músicas encontradas.
🎬 The Secret Life of Walter Mitty: 12 músicas encontradas.
🎬 Warm Bodies: 17 músicas encontradas.
🎬 Hugo: 15 músicas encontradas.
🎬 Before Midnight: 3 músicas encontradas.

In [8]:
yet_another_film_list = [
    "About Schmidt", "A Serious Man", "All the Real Girls", "American Animals", "American Splendor", "An Education", "Another Earth", "Antwone Fisher", "August Rush", "Babel",
    "Barry", "Being Flynn", "Being Julia", "Bella", "Benny & Joon", "Big Eyes", "Billy Elliot", "Blue Jay", "Blue Valentine", "Broken Flowers",
    "Cadillac Records", "Captain Fantastic", "Celeste and Jesse Forever", "Charlie Wilson's War", "Chasing Mavericks", "Cold Mountain", "Crazy Heart", "Cyrano", "Danny Collins", "Days of Heaven",
    "Dead Poets Society", "Dear Zachary", "Demolition", "Detroit", "Dope", "Down in the Valley", "Dreamgirls", "Eight Grade", "End of the Tour", "Evening",
    "Everything Must Go", "Factory Girl", "Frank", "Garden State", "Georgia Rule", "Get Low", "Girl, Interrupted", "Girl Most Likely", "Goodbye Christopher Robin", "Hearts Beat Loud",
    "High Fidelity", "Hope Springs", "I'm Not There", "In America", "In the Mood for Love", "Infinitely Polar Bear", "Into the Wild", "It’s Kind of a Funny Story", "Jeff, Who Lives at Home", "Junebug",
    "Just Like Heaven", "Keeping the Faith", "King Richard", "Last Night", "Lean on Pete", "Leaving Las Vegas", "Like Sunday, Like Rain", "Lion", "Little Women (2019)", "London Boulevard",
    "Love and Other Drugs", "Love Happens", "Machine Gun Preacher", "Maggie’s Plan", "Mandy", "Manic", "Marie Antoinette", "Martha Marcy May Marlene", "Mona Lisa Smile", "Music and Lyrics",
    "My Blueberry Nights", "Never Let Me Go", "Nowhere Boy", "On Chesil Beach", "Only Yesterday", "Our Brand Is Crisis", "Paris, je t'aime", "Pieces of April", "Puzzle", "Red Rocket",
    "Remember the Titans", "Running with Scissors", "Rust and Bone", "Secretariat", "Serena", "Short Term 12", "Sing", "Skate Kitchen", "Sleeping with Other People", "Stranger than Fiction"
]
analisar_trilhas_thread(yet_another_film_list, 'trilhas_sonoras_gpt9.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: About Schmidt

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Barry

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Cadillac Records

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Dead Poets Society

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Everything Must Go

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: High Fidelity

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Just Like Heaven

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Love and Other Drugs

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: My Blueberry Nights

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Remember the Titans
🎬 Everything Must Go: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Factory Girl
🎬 Barry: 25 músicas encontradas.
🎬 Cadillac Records: 0 músicas enco

In [9]:
eighth_movie_list = [
    "20th Century Women", "A Ghost Story", "A Hologram for the King", "A Man Called Ove", "A Most Violent Year", "A Single Man", "A Walk to Remember", "All Is Lost", "American Honey", "Anomalisa",
    "Arthur Newman", "Away from Her", "Beasts of the Southern Wild", "Before I Fall", "Beirut", "Big Fish", "Blinded by the Light", "Blood Diamond", "Broken", "Can You Ever Forgive Me?",
    "Candy", "Captain Corelli’s Mandolin", "Cassandra’s Dream", "Charlie Countryman", "Cinderella Man", "Closer", "Colette", "Cracks", "Creation", "Cry Macho",
    "Dear Frankie", "Disobedience", "Dogville", "Don't Worry, He Won’t Get Far on Foot", "Down to the Bone", "Eastern Promises", "Ella Enchanted", "Enduring Love", "Even Cowgirls Get the Blues", "Far from Heaven",
    "Five Feet Apart", "Focus", "For Colored Girls", "Frida", "Frozen River", "Get Real", "Girlhood", "God’s Pocket", "Good Kill", "Great Expectations (2012)",
    "Happythankyoumoreplease", "Harry Brown", "Heaven Knows What", "Her Smell", "High Art", "Higher Ground", "I Am Love", "I Origins", "I'm Your Woman", "In Her Shoes",
    "Irreplaceable You", "It Felt Like Love", "Jackie", "Jane Eyre (2011)", "Journey to the End of the Night", "Kaboom", "Kate & Leopold", "Kill Your Darlings", "Kinsey", "Laggies",
    "Land", "Last Days", "Last Flag Flying", "Liberal Arts", "Like Father", "Little Accidents", "Little Fish", "Love and Mercy", "Love is Strange", "Lucky",
    "Martha Marcy May Marlene", "Max Richter’s Sleep", "Miss Pettigrew Lives for a Day", "My Summer of Love", "Neruda", "Never Goin’ Back", "Night Moves", "On the Road", "Only the Brave", "Outside In",
    "Paper Man", "Paris Can Wait", "Peaceful Warrior", "People Places Things", "Phoebe in Wonderland", "Pieces of a Woman", "Please Stand By", "Private Life", "Queen of Katwe", "Rachel Getting Married"
]
analisar_trilhas_thread(eighth_movie_list, 'trilhas_sonoras_gpt10.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: 20th Century Women

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Arthur Newman

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Candy

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Dear Frankie

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Five Feet Apart

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Happythankyoumoreplease

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Irreplaceable You

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Land

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Martha Marcy May Marlene

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Paper Man
🎬 Happythankyoumoreplease: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Harry Brown
🎬 Irreplaceable You: 0 músicas encontradas.
⚠ Nenhuma música encontrada para est

In [10]:
ninth_movie_list = [
    "A Beautiful Mind", "A Scanner Darkly", "A United Kingdom", "Adore", "After the Wedding", "All Good Things", "An American Crime", "Angela's Ashes", "Another Year", "Apocalypse Now",
    "At Eternity's Gate", "Autumn Sonata", "Beautiful Creatures", "Before the Devil Knows You're Dead", "Being There", "Beneath the Harvest Sky", "Beyond the Lights", "Big Sur", "Black Snake Moan", "Blindness",
    "Boys Don't Cry", "Bright Star", "Broken Embraces", "Brothers", "Camp X-Ray", "Carnage", "Chloe", "Christine", "Chronically Metropolitan", "Conviction",
    "Coriolanus", "Crazy/Beautiful", "Crimson Peak", "Cut Bank", "Dancer in the Dark", "Dark Places", "Deep Water", "Desert Flower", "Detachment", "Dirty Girl",
    "Don’t Come Knocking", "Downsizing", "Eden", "El Club", "El Secreto de Sus Ojos", "Electrick Children", "Elizabeth: The Golden Age", "Enemy", "Everest", "Factory Girl",
    "Farewell My Queen", "Fat Girl", "Fish Tank", "First Reformed", "Frankie", "Freeheld", "Frozen (2013)", "Gainsbourg: A Heroic Life", "Giant Little Ones", "Girl Asleep",
    "God Help the Girl", "Good Bye Lenin!", "Gretel & Hansel", "Half Nelson", "Happening", "Harold and Maude", "Heavenly Creatures", "Hello I Must Be Going", "Henry's Crime", "Honey Boy",
    "Hounds of Love", "How I Live Now", "I Am David", "In Bloom", "Incendies", "Into the Forest", "Io Sono L'Amore", "It Comes at Night", "Jack Goes Boating", "Journey's End",
    "Kajillionaire", "Killing Them Softly", "Kiss Kiss Bang Bang", "Lamb", "Last Night in Soho", "Lions for Lambs", "Louder Than Bombs", "Love Sonia", "Made in Dagenham", "Mala Mala",
    "Man on Wire", "Margaret", "Margin Call", "Mary and Max", "Men, Women & Children", "Mid90s", "Monos", "Morris from America", "Mother!", "Mozart and the Whale"
]
analisar_trilhas_thread(ninth_movie_list, 'trilhas_sonoras_gpt11.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: A Beautiful Mind

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: At Eternity's Gate

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Boys Don't Cry

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Coriolanus

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Don’t Come Knocking

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Farewell My Queen

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: God Help the Girl

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Hounds of Love

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Kajillionaire

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Man on Wire
🎬 Man on Wire: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Margaret
🎬 Don’t Come Knocking: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este fi

In [11]:
tenth_movie_list = [
    "A Most Wanted Man", "A Prayer Before Dawn", "A Simple Favor", "A Tale of Love and Darkness", "Ad Astra", "After Yang", "Alice Doesn't Live Here Anymore", "All the Pretty Horses", "Amores Perros", "Animals",
    "Anthropoid", "As It Is in Heaven", "Bad Education", "Beautiful Boy", "Becoming Jane", "Before We Go", "Being 17", "Belle", "Bernie", "Breathe",
    "Buffalo '66", "Calvary", "Capernaum", "Captive State", "Certified Copy", "Charlie Says", "Cloud Atlas", "Colossal", "Come and See", "Control",
    "Cow", "Crimson Gold", "Custody", "Dear White People", "Desert Hearts", "Dogtooth", "Dom Hemingway", "Downfall", "Dreamland", "Edge of the World",
    "Elles", "End of Watch", "Eye in the Sky", "Faces Places", "Falling", "Farewell Amor", "Fatale", "Felicite", "Firecrackers", "First They Killed My Father",
    "Foxcatcher", "Frantz", "Funny Games", "Ghost World", "Girl", "Glassland", "Godland", "Gold", "Good Posture", "Great Freedom",
    "Guilty of Romance", "Handsome Devil", "Happily", "Here and Now", "Hesher", "Hit the Road", "Holy Motors", "House of Sand and Fog", "Hunger", "Hustle & Flow",
    "I Am Sam", "I Killed My Mother", "I, Daniel Blake", "I'm Thinking of Ending Things", "In Fabric", "In the Bedroom", "Intimacy", "It Follows", "Joe", "Julieta",
    "Just Mercy", "Kinetta", "Krisha", "Leave No Trace", "Let Me In", "Let the Right One In", "Life Itself", "Lingua Franca", "Little Ashes", "Living",
    "London to Brighton", "Loving", "Luce", "Lucy in the Sky", "Man Push Cart", "Marguerite", "Martha Marcy May Marlene", "Me and Earl and the Dying Girl", "Mediterranea", "Memento"
]
analisar_trilhas_thread(tenth_movie_list, 'trilhas_sonoras_gpt12.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: A Most Wanted Man

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Anthropoid

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Buffalo '66

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Cow

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Elles

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Foxcatcher

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Guilty of Romance

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: I Am Sam

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Just Mercy

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: London to Brighton
🎬 London to Brighton: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Loving
🎬 Guilty of Romance: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: 

In [12]:
eleventh_movie_list = [
    "127 Hours", "5 to 7", "A Brighter Summer Day", "A Perfect World", "Aftersun", "Ali", "All Is True", "All the Old Knives", "American Made", "Annihilation",
    "Antlers", "Archive", "Atlantics", "Barfly", "Before the Rains", "Begin Again", "Being Frank", "Ben Is Back", "Big Night", "Birdy",
    "Blaze", "Blue Caprice", "Boiling Point", "Boy A", "Brimstone", "Broken City", "Broken English", "By the Sea", "Captain Abu Raed", "Caught",
    "Certain Women", "Clouds of Sils Maria", "Code 46", "Concussion", "Cosmopolis", "Courage Under Fire", "C'mon C'mon", "Creep", "Cure", "Cymbeline",
    "Dakota Skye", "Dark Waters", "Daughter of Mine", "Delicatessen", "Departure", "Des hommes et des dieux", "Don't Look Now", "Down to You", "Drift", "Driveways",
    "Eat Pray Love", "Elena", "Embrace of the Serpent", "End of Sentence", "Enter the Void", "Euphoria", "Even the Rain", "Far from the Madding Crowd", "Fast Color", "Flee",
    "Four Good Days", "Foxfire", "Gerry", "Giant", "Girl with a Pearl Earring", "God's Own Country", "Gosford Park", "Graduation", "Great Expectations (1998)", "Greenberg",
    "Gretel & Hansel", "Happy Together", "Harakiri", "Harmonium", "Heartbeats", "Hello My Name Is Doris", "High Life", "His House", "Hunger", "Hustlers",
    "I Am Love", "I Don't Feel at Home in This World Anymore", "I Lost My Body", "I Smile Back", "Ida", "In a Better World", "In the Cut", "Indian Horse", "Inside I'm Dancing", "Irina Palm",
    "Jojo Rabbit", "Journal for Jordan", "Joy", "Julie & Julia", "Keep the Lights On", "Kubo and the Two Strings", "La Ciénaga", "La La Land", "Last Train Home", "Late Night"
]
analisar_trilhas_thread(eleventh_movie_list, 'trilhas_sonoras_gpt13.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: 127 Hours

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Antlers

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Blaze

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Certain Women

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Dakota Skye

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Eat Pray Love

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Four Good Days

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Gretel & Hansel

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: I Am Love

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Jojo Rabbit
🎬 Dakota Skye: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Dark Waters
🎬 Certain Women: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Clouds o

# Ainda mais filmes

In [5]:
twelfth_movie_list = [
    "A Ghost Story", "A Love Song", "A Perfect Day", "About Ray", "Across the Universe", "Ali & Ava", "All the Real Girls", "American Animals", "American Honey", "American Pastoral",
    "And Then We Danced", "Another Earth", "Apples", "At Any Price", "Away We Go", "Beasts of the Southern Wild", "Before I Fall", "Berlin Syndrome", "Big Eyes", "Billy Lynn's Long Halftime Walk",
    "Black Bear", "Blackbird", "Blindspotting", "Blue Jay", "Blue Valentine", "Body Heat", "Boogie", "Broken Flowers", "Buffalo Soldiers", "Cairo Time",
    "Call Jane", "Cemetery Junction", "Charlie Bartlett", "Cherry", "Christine (2016)", "Cigarette Burns", "Cold War", "Columbus", "Congo", "Control Room",
    "Country Strong", "Coyote Ugly", "Creep 2", "Crutch", "Cuban Fury", "Daddy Longlegs", "Damsel", "Dear Zachary", "Demolition", "Desperados",
    "Devil's Knot", "Disobedience", "Don Jon", "Don't Worry, He Won't Get Far on Foot", "Dreamgirls", "Duck Butter", "Eighth Grade", "Enduring Love", "Everybody's Everything", "Ex Machina",
    "Fifty Dead Men Walking", "Final Portrait", "Fleeing by Night", "For Ellen", "For the Boys", "Frances Ha", "Free Solo", "Funny Cow", "Giant (2017)", "Girlhood",
    "God Bless America", "Godzilla (2014)", "Good Time", "Gook", "Gran Torino", "Greetings from Tim Buckley", "Grizzly Man", "Happythankyoumoreplease", "Harriet", "Hearts Beat Loud",
    "Her Smell", "Herself", "High Flying Bird", "Honeyland", "Hope Gap", "Hotel Mumbai", "House of Gucci", "Howards End", "Howl", "I Am Not a Witch",
    "I Origins", "I Used to Go Here", "I'm Not There", "In Her Shoes", "In the Fade", "In the Mood for Love", "Infinitely Polar Bear", "Inheritance", "Inside Man", "Into the Wild"
]
analisar_trilhas_thread(twelfth_movie_list, 'trilhas_sonoras_gpt14.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: A Ghost Story

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: And Then We Danced

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Black Bear

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Call Jane

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Country Strong

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Devil's Knot

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Fifty Dead Men Walking

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: God Bless America

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Her Smell

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: I Origins
🎬 God Bless America: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Godzilla (2014)
🎬 Call Jane: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Anali

In [6]:
thirteenth_movie_list = [
    "A Man Called Ove", "A Most Violent Year", "A Single Man", "A Street Cat Named Bob", "About Time", "Adrift", "All Is Lost", "Amistad", "Amy", "An American Tail",
    "Another Round", "Antwone Fisher", "August: Osage County", "Babel", "Band Aid", "Beasts Clawing at Straws", "Being Flynn", "Belfast", "Bernard and Huey", "Black Gold",
    "Blood Diamond", "Blue Is the Warmest Color", "Bottle Rocket", "Boundaries", "Boychoir", "Breaking and Entering", "Brick", "Brightburn", "Bronson", "Brooklyn",
    "Cake", "Can You Ever Forgive Me?", "Captain Fantastic", "Carol", "Catch and Release", "Charlie Wilson's War", "Children of Men", "Christine (1983)", "Clemency", "Closer",
    "Cold in July", "Coming Home", "Conversations with Other Women", "Cooley High", "Cowboys", "Crazy Heart", "Crisis", "Crossing Over", "Cry Macho", "Cured",
    "Dallas Buyers Club", "Dark River", "Darkest Hour", "Dear Frankie", "Departure (2015)", "Detour", "Diabolique", "Dogman", "Don't Think Twice", "Down Terrace",
    "Dreaming Walls", "Drive My Car", "Dying Young", "Eagle vs Shark", "Earthquake Bird", "Eastern Promises", "Eat That Question", "Echo in the Canyon", "Edison", "Elle",
    "Enduring Love", "Enemy at the Gates", "Enter Nowhere", "Everything Must Go", "Evil", "Exit Through the Gift Shop", "Faces in the Crowd", "Fading Gigolo", "Fanny and Alexander", "Far North",
    "Fast Food Nation", "Fearless", "Felon", "Finding Neverland", "Fireflies in the Garden", "Flipped", "Florence Foster Jenkins", "Fluke", "Four Lions", "Frank",
    "Freedom Writers", "Fresh", "Fruitvale Station", "Fur", "Gacy", "Gangs of New York", "Garden State", "Gerry", "Ghosts of Girlfriends Past", "Girl Rising"
]
analisar_trilhas_thread(thirteenth_movie_list, 'trilhas_sonoras_gpt15.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: A Man Called Ove

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Another Round

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Blood Diamond

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Cake

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Cold in July

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Dallas Buyers Club

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Dreaming Walls

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Enduring Love

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Fast Food Nation

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Freedom Writers
🎬 Dreaming Walls: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Drive My Car
🎬 A Man Called Ove: 7 músicas encontradas.
🎬 Cold in July: 11 músicas encontradas.
🎬 Blood Dia

In [7]:
fourteenth_movie_list = [
    "A Time for Drunken Horses", "A Woman Under the Influence", "Adventureland", "After the Storm", "All About Lily Chou-Chou", "All Eyez on Me", "American Splendor", "American Violet", "Angel-A", "Anomalisa",
    "As Good as It Gets", "Ash Is Purest White", "Atonement", "Attenberg", "Autumn in New York", "Bad Lieutenant: Port of Call New Orleans", "Bal", "Band of Robbers", "Bang Gang", "Baran",
    "Beautiful Thing", "Before Midnight", "Belleville Rendez-vous", "Beyond the Black Rainbow", "Big Fish", "Black '47", "Black Book", "Blind", "Blow", "Blood Simple",
    "Blue Car", "Body of Lies", "Boogie Nights", "Boyhood", "Breakdown", "Breaking the Waves", "Breathless", "Brick Lane", "Brigsby Bear", "Brother",
    "Burning", "Cake (2014)", "Cam", "Candy", "Captive", "Carandiru", "Cashback", "Cassandra's Dream", "Celeste and Jesse Forever", "Changeling",
    "Chasing Ice", "Cherry Blossoms", "Children of Heaven", "Chocolat", "Choke", "Cinema Paradiso", "Circle of Friends", "Citizen Ruth", "Clean", "Clerks",
    "Cold Comes the Night", "Cold Pursuit", "Come Undone", "Confessions of a Dangerous Mind", "Control (2004)", "Coraline", "Coup de Torchon", "Crazy", "Crimson Tide", "Crisis (2021)",
    "Crouching Tiger, Hidden Dragon", "Crows Zero", "Cry-Baby", "Cure (1997)", "Custody (2017)", "Cyrano", "Daddy's Little Girls", "Dancer", "Dancer and the Thief", "Dark Blue",
    "Dark Days", "Dark Horse", "Dark Skies", "Darling", "Dead Poets Society", "Dear Dictator", "Deep Blue Sea", "Delirious", "Denial", "Detroit",
    "Diary of a Teenage Girl", "Die Welle", "Diving Bell and the Butterfly", "Djinn", "Dog Soldiers", "Donnie Brasco", "Double Lover", "Downfall (2004)", "Downsized", "Doubt"
]
analisar_trilhas_thread(fourteenth_movie_list, 'trilhas_sonoras_gpt16.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: A Time for Drunken Horses

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: As Good as It Gets

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Beautiful Thing

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Blue Car

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Burning

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Chasing Ice

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Cold Comes the Night

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Crouching Tiger, Hidden Dragon

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Dark Days

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Diary of a Teenage Girl
🎬 A Time for Drunken Horses: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: A Woman Under the Influence
🎬 Burning: 0 músicas encontradas.
⚠ 

In [8]:
fifteenth_movie_list = [
    "20th Century Women", "A Fantastic Woman", "A Hidden Life", "A Monster Calls", "A Shaun the Sheep Movie: Farmageddon", "A Simple Plan", "A Tale of Love and Darkness", "Abandon", "About Schmidt", "After Life",
    "Alamar", "All Good Things", "All the Invisible Children", "All the King's Men", "All Together Now", "American Fable", "American History X", "American Outlaws", "An Education", "An Impossible Project",
    "Angel Heart", "Another Happy Day", "Antonement", "Any Day Now", "Ararat", "Are You There God? It's Me, Margaret", "Argo", "Around the Bend", "As It Is in Heaven", "Ask the Dust",
    "Assassination of Jesse James by the Coward Robert Ford", "August Rush", "Autumn Sonata", "Awakenings", "Away from Her", "Babette's Feast", "Backbeat", "Bad Education", "Bad Times at the El Royale", "Ballast",
    "Baraka", "Barry Lyndon", "Bart Got a Room", "Basquiat", "Battle of the Sexes", "Beautiful Boy", "Bee Season", "Being Julia", "Being There", "Benny & Joon",
    "Beyond Rangoon", "Big Bad Wolves", "Bird", "Birth", "Black and White", "Black Sea", "Blame", "Blaze", "Bless Me, Ultima", "Blood and Wine",
    "Blown Away", "Blue Story", "Blue Thunder", "Bluebeard", "Bob Roberts", "Bombshell", "Bonnie and Clyde", "Bound", "Boyz n the Hood", "Breakfast on Pluto",
    "Breathe", "Bright Star", "Broken Embraces", "Bubba Ho-Tep", "Buddha Collapsed out of Shame", "Buffalo '66", "Bunny and the Bull", "Burnt", "But I'm a Cheerleader", "Café de Flore",
    "Cairo Station", "Call Me by Your Name", "Camp X-Ray", "Captain Corelli's Mandolin", "Caramel", "Carriers", "Catch Me If You Can", "Celda 211", "Chaplin", "Charlie Says",
    "Che", "Chéri", "Chi-Raq", "Children of a Lesser God", "Chungking Express", "Cinderella Man", "Circumstance", "City of Angels", "Claire's Knee", "Clean, Shaven"
]
analisar_trilhas_thread(fifteenth_movie_list, 'trilhas_sonoras_gpt17.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: 20th Century Women

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Alamar

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Angel Heart

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Assassination of Jesse James by the Coward Robert Ford

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Baraka

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Beyond Rangoon

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Blown Away

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Breathe

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Cairo Station

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Che
🎬 Cairo Station: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Call Me by Your Name
🎬 20th Century Women: 22 músicas encontradas.
🎬 Breathe: 0 músicas encontradas.

In [9]:
sixteenth_movie_list = [
    "The Last Duel", "Triangle of Sadness", "Tinker Tailor Soldier Spy", "The Vast of Night", "Under the Silver Lake", "Uncut Gems", "The Unbearable Weight of Massive Talent", "The United States vs. Billie Holiday", "Tombstone", "Unfaithful",
    "Lars and the Real Girl", "La Vie en Rose", "Leave No Trace", "Loving", "Little Miss Sunshine", "Lemon", "Let Me In", "Lion", "Locke", "Looper",
    "Velvet Goldmine", "Valhalla Rising", "Vera Drake", "Venus", "Vox Lux", "Veronica", "Vivarium", "Victoria", "Volver", "Violet & Daisy",
    "Zodiac", "Zero Dark Thirty", "Z for Zachariah", "Zama", "Zoe", "Zookeeper's Wife", "Zola", "Zombieland", "Zombieland: Double Tap", "Zen",
    "Nightcrawler", "Nocturnal Animals", "No Country for Old Men", "Never Let Me Go", "Night Moves", "Nomadland", "Neruda", "Nebraska", "Notes on a Scandal", "Not Okay",
    "Moonlight", "Manchester by the Sea", "Memento", "Martha Marcy May Marlene", "Mother!", "Midnight Special", "Mud", "Minding the Gap", "Monos", "Maggie's Plan",
    "This Must Be the Place", "Take Shelter", "The Killing of a Sacred Deer", "The Florida Project", "The Place Beyond the Pines", "The Painted Veil", "The Immigrant", "The Fall", "The Road", "The Machinist",
    "Upgrade", "Unbreakable", "Us", "Underworld", "Ultraviolet", "Uncle Frank", "Untamed Heart", "Ulee's Gold", "Unstrung Heroes", "Unhook the Stars",
    "Loving Vincent", "Last Night in Soho", "Legend", "Leave Her to Heaven", "Laggies", "Land of Mine", "Let the Right One In", "Love & Mercy", "Lock, Stock and Two Smoking Barrels", "Lost in the Desert",
    "Voyagers", "Valentine", "Vampire's Kiss", "Vanilla Sky", "Venom", "Vicky Cristina Barcelona", "Vincere", "Violet", "V/H/S", "V/H/S/2"
]
analisar_trilhas_thread(sixteenth_movie_list, 'trilhas_sonoras_gpt18.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: The Last Duel

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Lars and the Real Girl

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Velvet Goldmine

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Zodiac

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Nightcrawler

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Moonlight

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: This Must Be the Place

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Upgrade

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Loving Vincent

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Voyagers
🎬 This Must Be the Place: 17 músicas encontradas.
🎬 Voyagers: 1 músicas encontradas.
🎬 Nightcrawler: 32 músicas encontradas.
🎬 Velvet Goldmine: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilh

In [10]:
seventeenth_movie_list = [
    "The Trial of the Chicago 7", "The Tender Bar", "Titane", "Things Heard & Seen", "The Tree of Life", "The Third Man", "The Theory of Everything", "The Town", "The Tragedy of Macbeth", "The Tale",
    "Laggies", "Land", "Lemonade Mouth", "La La Land", "Little Woods", "Lean on Pete", "Like Crazy", "Luce", "Love", "Last Flag Flying",
    "Vice", "Vicky", "Valley Girl", "Very Good Girls", "Vampires vs. Bronx", "Vanguard", "Vera", "Venus in Fur", "Violent Night", "V/H/S/94",
    "Zachariah", "Zindagi Na Milegi Dobara", "Zero Days", "Zift", "Zatoichi", "Zindagi 50-50", "Zaytoun", "Zarafa", "Zombiology: Enjoy Yourself Tonight", "Zach Galifianakis: Live at the Purple Onion",
    "Naked", "Nowhere Boy", "No One Knows About Persian Cats", "Notorious", "Nostalgia", "Nineteen Eighty-Four", "Nanny McPhee", "Nina Forever", "Never Rarely Sometimes Always", "Night at the Museum",
    "Memories of Murder", "Me and Earl and the Dying Girl", "Melancholia", "Midnight in Paris", "Mirror", "Mystic River", "My Own Private Idaho", "My Summer of Love", "Marie Antoinette", "Minari",
    "Timecrimes", "Tusk", "Take Me Home Tonight", "The Turin Horse", "The Terminal", "The Secret Life of Walter Mitty", "The Spectacular Now", "The Station Agent", "The Sweet Hereafter", "The Swimmer",
    "Uptown Girls", "Uncle Boonmee Who Can Recall His Past Lives", "Unrest", "Us and Them", "Urban Hymn", "Unconscious", "U-571", "Under the Same Moon", "Udaan", "Unfreedom",
    "Love Simon", "Long Shot", "Life Itself", "Legend of the Fall", "Louder Than Bombs", "Love Happens", "Lucky Number Slevin", "Lords of Dogtown", "London Boulevard", "Life of Pi",
    "Viy", "Vaya", "Vampyr", "Vincent", "Voice from the Stone", "Visages Villages", "Victor Frankenstein", "Vampire Academy", "Vox", "Vampires"
]
analisar_trilhas_thread(seventeenth_movie_list, 'trilhas_sonoras_gpt19.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: The Trial of the Chicago 7

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Laggies

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Vice

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Zachariah

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Naked

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Memories of Murder

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Timecrimes

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Uptown Girls

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Love Simon

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Viy
🎬 Vice: 11 músicas encontradas.
🎬 Love Simon: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Long Shot
🎬 Naked: 13 músicas encontradas.
🎬 Zachariah: 0 músicas encontradas.
⚠ Nenhuma música encontrada pa

In [11]:
eighteenth_movie_list = [
    "The Way Back", "The Whale", "The Witch", "The Wind Rises", "The Wolf of Wall Street", "The Wonder", "The Words", "The World's Fastest Indian", "The Worst Person in the World", "The Wailing",
    "Little Children", "Lamb", "Last Days", "Les Misérables", "Leave It on the Floor", "Life During Wartime", "Like Father, Like Son", "Lorenzo's Oil", "Love Song", "Lost in Translation",
    "Vesper", "Valentine's Day", "Valiant", "Velvet Buzzsaw", "Veronica Guerin", "Vice Versa", "View from the Top", "Vincenzo", "Volcano", "V/H/S Viral",
    "Zebrahead", "Zombieland Saga", "Zorba the Greek", "Zombiesthan", "Zashchitniki", "Zindagi Kitni Haseen Hay", "Zift", "Zhou Yu's Train", "Zatoichi's Cane Sword", "Zelary",
    "Neruda", "Not One Less", "No Man's Land", "Now Is Good", "Nausicaä of the Valley of the Wind", "Ned Rifle", "Never Cry Wolf", "Nobody", "North Country", "November",
    "Man on the Moon", "Margin Call", "Mister Lonely", "Moon", "Moonrise Kingdom", "Monsoon Wedding", "My Winnipeg", "My Life Without Me", "Mystery Road", "Madeline's Madeline",
    "To the Bone", "Take This Waltz", "The Voices", "The Wrestler", "The Way", "The Wind That Shakes the Barley", "The Wife", "The Young Victoria", "The Yellow Handkerchief", "The Yards",
    "Urban Cowboy", "Under Fire", "Ultrasonic", "Uptown Saturday Night", "Untraceable", "Unthinkable", "Upstream Color", "Undine", "Us Kids", "Unicorn Store",
    "Loving Pablo", "Love Streams", "Life Is Sweet", "Laggies", "Lords of Chaos", "Love & Basketball", "Like Water for Chocolate", "Lean on Me", "Late Spring", "Lady Vengeance",
    "V/H/S/99", "Vampire Circus", "Velvet Underground", "Viktoria", "View from a Blue Moon", "Victor Victoria", "Vinyan", "Voices", "Vortex", "Vampires: Los Muertos"
]
analisar_trilhas_thread(eighteenth_movie_list, 'trilhas_sonoras_gpt20.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: The Way Back

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Little Children

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Vesper

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Zebrahead

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Neruda

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Man on the Moon

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: To the Bone

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: Urban Cowboy

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Loving Pablo

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: V/H/S/99
🎬 Zebrahead: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Zombieland Saga
🎬 Neruda: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Not One Less
E

Traceback (most recent call last):
  File "/tmp/ipykernel_181855/4094389521.py", line 24, in buscar_trilha_soundtracknet_bs4
    response.raise_for_status()
  File "/home/lutero/Projetos/machine_learning/lib/python3.12/site-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 404 Client Error: Not Found for url: https://www.soundtrack.net/album/invalid_album.php


🎬 The Way Back: 6 músicas encontradas.
🎬 To the Bone: 12 músicas encontradas.
🎬 Man on the Moon: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Margin Call
🎬 Loving Pablo: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Love Streams
🎬 Not One Less: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: No Man's Land
🎬 Zombieland Saga: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Zorba the Greek
🎬 Little Children: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Lamb
🎬 Urban Cowboy: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Under Fire
🎬 Love Streams: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Life Is Sweet
🎬 Margin Call: 0

In [12]:
nineteenth_movie_list = [
    "The Little Things", "The Light Between Oceans", "The Legend of Tarzan", "The Lost Daughter", "The Love Witch", "The Last King of Scotland", "The Long Goodbye", "The Lobster", "The Ledge", "The Ladykillers",
    "Underground", "Unplanned", "Uninhabited", "Under Suspicion", "Unknown", "Under the Tree", "Up for Love", "Useless", "Urbanized", "Unstoppable",
    "Living in Oblivion", "Life Happens", "Little Fish", "Love and Death", "Lucky", "Life is a Miracle", "Late Night", "Leviathan", "Lantana", "Lambada",
    "Veronika Voss", "Velvet", "Violeta Went to Heaven", "Voyage of Time", "Veronique and the Bleeding Hearts", "V/H/S/85", "Vinyl", "Valentino", "Violet Hour", "Vice Squad",
    "Zama", "Zardoz", "Zatôichi monogatari", "Zipper", "Zone 414", "Zulu", "Zamaana Deewana", "Zanjeer", "Zion", "Zai Zai Zai Zai",
    "Nightwatch", "Not Reconciled", "No Direction Home", "Nothing But the Truth", "Nobody Knows", "Night and the City", "Nina", "No Man of God", "Nights of Cabiria", "Nowhere",
    "My Left Foot", "Manhunter", "Melvin and Howard", "Memories", "My Brilliant Career", "Maniac", "Man from Reno", "Mid90s", "Mondo Trasho", "Mystic Pizza",
    "The Tale of Princess Kaguya", "The Thirteenth Floor", "The Trial", "The Third Murder", "The Turning", "The Two Faces of January", "The Tenth Man", "The Truth About Cats & Dogs", "The Trouble with Harry", "The Tribe",
    "Uzak", "Up the Sandbox", "Umoja", "Uncle Buck", "Undercover Brother", "Unbearable Lightness of Being", "Untamed", "Urbania", "Une Femme est une Femme", "Unstrung Heroes",
    "Love and Monsters", "Last Night", "Life of Crime", "Little Men", "Lords of Discipline", "Luce", "Loving You", "Lilya 4-Ever", "Lies My Father Told Me", "Laggies"
]
analisar_trilhas_thread(nineteenth_movie_list, 'trilhas_sonoras_gpt21.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: The Little Things

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: Underground

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Living in Oblivion

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Veronika Voss

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Zama

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Nightwatch

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: My Left Foot

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: The Tale of Princess Kaguya

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Uzak

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Love and Monsters
🎬 Veronika Voss: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Velvet
🎬 Living in Oblivion: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisand

In [13]:
twentieth_movie_list = [
    "The Water Diviner", "The White Ribbon", "The War Zone", "The Way Way Back", "The Woman King", "The Witcher: Nightmare of the Wolf", "The Wild Goose Lake", "The Wackness", "The Wonders", "The War of the Worlds",
    "Lost Girls", "Leave", "Lords of Salem", "Love Serenade", "Last Orders", "Lust, Caution", "Life with Father", "Lou", "Love Me If You Dare", "Lucky Grandma",
    "Valley of Love", "Violet Perfume", "Vampire Hunter D", "Vapors", "Vertical Limit", "Villain", "Vampyros Lesbos", "Vacation", "Volcano High", "Violet Evergarden",
    "Zelary", "Zzyzx", "Zazie dans le Métro", "Zatoichi Meets Yojimbo", "Zodiac: Signs of the Apocalypse", "Zipperface", "Zombie Honeymoon", "Zameer", "Z Storm", "Zambezia",
    "Nothing Personal", "Never Goin' Back", "Nico, 1988", "Naked Lunch", "Naked Gun", "No Blade of Grass", "Nina's Tragedies", "Not One Less", "Nostalgia for the Light", "North",
    "My Name Is Tanino", "Mary and Max", "Man on Wire", "Meek's Cutoff", "Mr. Nobody", "Mortal Thoughts", "Ma Ma", "Margin for Error", "Mannequin", "Mindwalk",
    "That Obscure Object of Desire", "The Way Home", "The Wicker Tree", "The Woman in the Fifth", "The Young and Prodigious T.S. Spivet", "The Year My Voice Broke", "The Yakuza", "The Yellow Sea", "The Young Offenders", "The Young Girls of Rochefort",
    "UFO", "Under Fire", "Unforgiven", "Unmade Beds", "Ulysses' Gaze", "Ulzana's Raid", "Utz", "Uro", "Une liaison pornographique", "Urgh! A Music War",
    "Liar Liar", "Life Without Zoe", "Lost in America", "Liberty Heights", "Little Women", "Lola", "Loren Cass", "Live Flesh", "Life as a House", "Love is Strange"
]
analisar_trilhas_thread(twentieth_movie_list, 'trilhas_sonoras_gpt22.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: The Water Diviner

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: The War of the Worlds

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Love Me If You Dare

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Vacation

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Zombie Honeymoon

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: No Blade of Grass

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Mr. Nobody

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: The Woman in the Fifth

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Unforgiven

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: Life Without Zoe
🎬 Zombie Honeymoon: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Zameer
🎬 No Blade of Grass: 0 músicas encontradas.
⚠ Nenhuma música encontrada para

In [14]:
twenty_first_movie_list = [
    "The Girl with the Dragon Tattoo", "The Grey", "The Grandmaster", "The Great Beauty", "The Green Knight", "The Guilty", "The Ghost Writer", "The Goldfinch", "The Good Liar", "The Gift",
    "Little Odessa", "Lost in Beijing", "Le Doulos", "La Femme Nikita", "Love Exposure", "Last Tango in Paris", "Le Bonheur", "La Strada", "Lone Star", "L'Avventura",
    "Vera Cruz", "Vampire's Assistant", "Valhalla", "Violets Are Blue", "Veronika", "Volver a Empezar", "Vive L'Amour", "Valley Girl", "Vanishing Point", "Vessel",
    "Zan Boko", "Zatoichi's Vengeance", "Zero Bridge", "Ziddi", "Zama Zama", "Zapped!", "Zelly and Me", "Zone Rouge", "Zinzana", "Zegen",
    "No End in Sight", "Night and Fog", "Nocturne", "Nobody's Fool", "Not Easily Broken", "Nine Queens", "Notorious B.I.G.: Bigger Than Life", "Never on Sunday", "Narrow Margin", "Nerve",
    "Menace II Society", "Mouchette", "Mephisto", "Miss Julie", "Medea", "Mifune", "Midnight Cowboy", "Man of Marble", "Madeinusa", "Moolaade",
    "The Tunnel", "The Tree", "The Taste of Others", "The Teacher", "The Thief of Bagdad", "The Triplets of Belleville", "The Third Generation", "The Tall Man", "The Terrorizers", "The Thing Called Love",
    "Ulee's Gold", "Ugetsu", "Uncut Gems", "Un amour de jeunesse", "Under Electric Clouds", "Uncle Kent", "Underground Fragrance", "Undertow", "Unrelated", "Under the Skin",
    "Les Choristes", "La Haine", "Last Night in Paris", "Love in the Afternoon", "Love Exposure", "Lola Montès", "Late Autumn", "Last Train Home", "Let It Fall", "Let’s Scare Jessica to Death"
]
analisar_trilhas_thread(twenty_first_movie_list, 'trilhas_sonoras_gpt23.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: The Girl with the Dragon Tattoo

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: The Gift

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Lone Star

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: Valley Girl

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Zelly and Me

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Nine Queens

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Medea

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: The Teacher

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Uncut Gems

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: La Haine
🎬 La Haine: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Last Night in Paris
🎬 The Girl with the Dragon Tattoo: 20 músicas encontradas.
🎬 Lone Star: 0 músicas encontradas.
⚠ Nenhuma música

In [15]:
twenty_second_movie_list = [
    "The Pledge", "The Power of the Dog", "The Piano", "The Perfect Candidate", "The Perks of Being a Wallflower", "The Proposition", "The Paperboy", "The Past", "The Postman", "The Promise",
    "Let Him Go", "Lady Bird", "Last Holiday", "Lone Ranger", "Love and Other Drugs", "Like Someone in Love", "Little Voice", "Love Affair", "Love Song for Bobby Long", "Lionheart",
    "Vampire's Kiss", "Valhalla Rising", "Vampire Circus", "Voyager", "Veronica Mars", "Venus as a Boy", "Venus in Fur", "V/H/S: Viral", "Vesper", "Villa Amalia",
    "Zodiac Killer", "Zion and His Brother", "Zatoichi Challenged", "Zombieland: Double Tap", "Zombillénium", "Zhou Yu's Train", "Zero Kelvin", "Zatoichi and the Chess Expert", "Zookeeper", "Zero Theorem",
    "Nocturama", "Nausicaa", "North Face", "Noi the Albino", "Not Here to Be Loved", "Not the Messiah", "No Looking Back", "Not Since You", "Night of the Hunter", "No Place on Earth",
    "My Friend Dahmer", "My Life as a Dog", "My Summer of Love", "My Architect", "Mickey and the Bear", "Margarita with a Straw", "Mysterious Skin", "Mystery Men", "Marie Krøyer", "Melody",
    "The Secret in Their Eyes", "The Spirit of the Beehive", "The Selfish Giant", "The Seventh Continent", "The Stoning of Soraya M.", "The Sound of Insects", "The Savages", "The Scapegoat", "The Sleepwalker", "The Souvenir",
    "Unforgivable", "Under the Bombs", "Undertaking Betty", "Unreal Estate", "Uncanny", "Uncle Howard", "Urban Explorer", "Up the Down Staircase", "Untitled", "Us Kids",
    "Les Diaboliques", "La Ciénaga", "Le Samouraï", "L'âge d'or", "Last Resort", "Life According to Agfa", "Lily Tomlin: The Search for Signs of...", "La Notte", "Loulou", "Love Is the Devil"
]
analisar_trilhas_thread(twenty_second_movie_list, 'trilhas_sonoras_gpt24.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: The Pledge

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: The Promise

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Love Song for Bobby Long

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: V/H/S: Viral

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Zero Kelvin

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: Not the Messiah

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: Mickey and the Bear

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: The Seventh Continent

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Undertaking Betty

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: La Ciénaga
🎬 Undertaking Betty: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Unreal Estate
🎬 The Seventh Continent: 0 músicas encontradas.
⚠ Nenhuma música encontrad

In [16]:
twenty_third_movie_list = [
    "The Fall", "The Illusionist", "The Fountain", "The House of Sand and Fog", "The Black Dahlia", "The Handmaiden", "The Kite Runner", "The Proposition", "The Painted Bird", "The Dreamers",
    "Loro", "Let the Corpses Tan", "Louder Than Bombs", "La Dolce Vita", "La Ronde", "Loving Highsmith", "La Collectionneuse", "L’Amant", "Last Year at Marienbad", "Lenny",
    "Valmont", "Vernon, Florida", "Vampyr", "Valhalla", "Voice Over", "Vigil", "Vermeer: Master of Light", "V/H/S/94", "Venus and Serena", "Valentino: The Last Emperor",
    "Zaitochi: The Blind Swordsman", "Zazie", "Zamaana Deewana", "Zombies of Mora Tau", "Zazie dans le Métro", "Zero for Conduct", "Zach Galifianakis Live", "Zatoichi the Outlaw", "Z", "Zana",
    "Nausicaä", "Night of the Living Dead", "Night Watch", "No Man’s Land", "Nuit Blanche", "New York, I Love You", "Nanny McPhee Returns", "Nekromantik", "Nina of the Woods", "Naked Island",
    "Mishima", "Mothman Prophecies", "Men Don't Leave", "Mortal World", "My Joy", "Mysteries of Lisbon", "My King", "Marie-Jo and Her Two Lovers", "Miral", "Midnight Traveler",
    "The Secret World of Arrietty", "The Spanish Apartment", "The Swarm", "The Sea Inside", "The Secret Life of Words", "The Swell Season", "The Sessions", "The Signal", "The Smell of Us", "The Star",
    "Undefeated", "Uncut Gems", "Une Histoire de Fou", "Uncle Nino", "Under the Rainbow", "Untamed Heart", "Unrest", "Upstream Color", "Under the Sun", "Umi ga Kikoeru",
    "Les Misérables", "L'Enfant", "L'Atalante", "Lamerica", "Love Streams", "La Belle Noiseuse", "Little Children", "Luna", "Léolo", "Lars and the Real Girl"
]
analisar_trilhas_thread(twenty_third_movie_list, 'trilhas_sonoras_gpt25.csv')


🔄 Analisando parte 1 de 10...

🎥 Analisando trilha sonora de: The Fall

🔄 Analisando parte 2 de 10...

🎥 Analisando trilha sonora de: The Dreamers

🔄 Analisando parte 3 de 10...

🎥 Analisando trilha sonora de: Last Year at Marienbad

🔄 Analisando parte 4 de 10...

🎥 Analisando trilha sonora de: V/H/S/94

🔄 Analisando parte 5 de 10...

🎥 Analisando trilha sonora de: Zach Galifianakis Live

🔄 Analisando parte 6 de 10...

🎥 Analisando trilha sonora de: New York, I Love You

🔄 Analisando parte 7 de 10...

🎥 Analisando trilha sonora de: My Joy

🔄 Analisando parte 8 de 10...

🎥 Analisando trilha sonora de: The Sea Inside

🔄 Analisando parte 9 de 10...

🎥 Analisando trilha sonora de: Une Histoire de Fou

🔄 Analisando parte 10 de 10...

🎥 Analisando trilha sonora de: L'Enfant
🎬 V/H/S/94: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

🎥 Analisando trilha sonora de: Venus and Serena
🎬 New York, I Love You: 0 músicas encontradas.
⚠ Nenhuma música encontrada para este filme.

# Junta csv

In [1]:
import pandas as pd

def juntar_csvs(lista_arquivos, caminho_saida='junta2dataset.csv'):
    df_geral = pd.DataFrame()
    
    with open(lista_arquivos, 'r') as f:
        for linha in f:
            nome_arquivo = linha.strip()
            if nome_arquivo:  # ignora linhas vazias
                try:
                    df = pd.read_csv(nome_arquivo)
                    # Concatena com o dataframe geral
                    df_geral = pd.concat([df_geral, df], ignore_index=True)
                except Exception as e:
                    print(f"Erro ao ler {nome_arquivo}: {e}")
    
    # Remove duplicatas baseadas nas colunas especificadas
    df_geral = df_geral.drop_duplicates(subset=['filme', 'titulo_musica', 'artista_principal'])
    
    # Salva o resultado final
    df_geral.to_csv(caminho_saida, index=False)
    print(f'Arquivo consolidado salvo como {caminho_saida}')

juntar_csvs('a.txt')

Erro ao ler acousticbrainz-tonal.csv: [Errno 2] No such file or directory: 'acousticbrainz-tonal.csv'


KeyError: Index(['artista_principal', 'titulo_musica', 'filme'], dtype='object')